# Tuning and training the models for standard-pHLA-score

Input are the per-peptide-position ref2015 features for complex, already generated in ../Featurization/rosettaPPPEnergies.csv

We tune the parameters in the 5-fold-crossvalidation setting.

In [18]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterSampler, cross_val_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from IPython.display import display
from scipy import stats
import _pickle as cPickle
import statistics
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

## Load the standard ref2015 features

### Full dataset

In [19]:
## 1 - load the energies
def pppene_to_array(tmp):
    tmp = tmp.replace("(", "")
    tmp = tmp.replace(")", "")
    tmp = tmp.strip("[]")
    tmp = tmp.replace(" ", "")
    tmp = tmp.replace("\n", ",")
    return np.fromstring(tmp, dtype=float, sep=", ").reshape(9,20)

ppp_ene = pd.read_csv("../Featurization/rosettaPPPEnergies.csv")
ppp_ene = ppp_ene[["allele", "peptide", "binder", "ba", "energies", "total_energy"]]
ppp_ene["energies"] = ppp_ene["energies"].apply(pppene_to_array)
ppp_ene

,allele,peptide,binder,ba,energies,total_energy
0,A0101,YLEQLHQLY,1,0.574375,"[[-9.37669305, 4.67802037, 8.54111444, 10.3367...",112.623867
1,A0101,HSERHVLLY,1,0.574375,"[[-7.89190954, 0.93707113, 10.06233605, 2.7158...",91.902185
2,A0101,MTDPEMVEV,1,0.574375,"[[-8.25236275, 10.56587939, 7.5793572, 1.14710...",146.451590
3,A0101,LTDFIREEY,1,0.574375,"[[-8.43720197, 10.21830335, 7.113905, 36.51672...",138.735082
4,A0101,LLDQRPAWY,1,0.574375,"[[-8.18944861, 12.37002534, 6.69837214, 8.0238...",142.756344
...,...,...,...,...,...,...
77576,C1601,QQTTTSFQN,0,0.000000,"[[-8.23468271, 4.14167228, 9.40648541, 38.0082...",128.349704
77577,C1601,QQVEQMEIP,0,0.000000,"[[-9.20362039, 22.89414572, 10.40234849, 35.70...",159.872992
77578,C1601,QQWQVFSAE,0,0.000000,"[[-8.46025926, 3.82365938, 10.25920191, 61.414...",97.152888
77579,C1601,QRCVVLRFL,0,0.000000,"[[-7.11160825, 1.35151526, 9.62650699, 20.2230...",116.714004


### Map the full dataset to the training set

In [20]:
#Load split
train_set = pd.read_csv("../Datasets/train_set.csv")
train_set = train_set[["allele", "peptide", "fileloc", "allele_type", "fold_num"]]

#Merge to form the training set
train_dataset = pd.merge(ppp_ene, train_set, on=["allele", "peptide"], suffixes=["", "_y"], how="inner")
train_dataset

,allele,peptide,binder,ba,energies,total_energy,fileloc,allele_type,fold_num
0,A0101,YLEQLHQLY,1,0.574375,"[[-9.37669305, 4.67802037, 8.54111444, 10.3367...",112.623867,/home/anja/Documents/jayvee_data/singleconf/al...,HLA-A,2.0
1,A0101,HSERHVLLY,1,0.574375,"[[-7.89190954, 0.93707113, 10.06233605, 2.7158...",91.902185,/home/anja/Documents/jayvee_data/singleconf/al...,HLA-A,0.0
2,A0101,MTDPEMVEV,1,0.574375,"[[-8.25236275, 10.56587939, 7.5793572, 1.14710...",146.451590,/home/anja/Documents/jayvee_data/singleconf/al...,HLA-A,0.0
3,A0101,LTDFIREEY,1,0.574375,"[[-8.43720197, 10.21830335, 7.113905, 36.51672...",138.735082,/home/anja/Documents/jayvee_data/singleconf/al...,HLA-A,1.0
4,A0101,LLDQRPAWY,1,0.574375,"[[-8.18944861, 12.37002534, 6.69837214, 8.0238...",142.756344,/home/anja/Documents/jayvee_data/singleconf/al...,HLA-A,1.0
...,...,...,...,...,...,...,...,...,...
69793,C1601,QQTTTSFQN,0,0.000000,"[[-8.23468271, 4.14167228, 9.40648541, 38.0082...",128.349704,/home/anja/Documents/COMP590P/C_decoys/confs/C...,HLA-C,4.0
69794,C1601,QQVEQMEIP,0,0.000000,"[[-9.20362039, 22.89414572, 10.40234849, 35.70...",159.872992,/home/anja/Documents/COMP590P/C_decoys/confs/C...,HLA-C,4.0
69795,C1601,QQWQVFSAE,0,0.000000,"[[-8.46025926, 3.82365938, 10.25920191, 61.414...",97.152888,/home/anja/Documents/COMP590P/C_decoys/confs/C...,HLA-C,1.0
69796,C1601,QRCVVLRFL,0,0.000000,"[[-7.11160825, 1.35151526, 9.62650699, 20.2230...",116.714004,/home/anja/Documents/COMP590P/C_decoys/confs/C...,HLA-C,2.0


In [21]:
# extracting features in training format
# and get the cross-validation iterator
def get_energies(X):
    ene = np.roll(X, 4, axis = 0)[:9,:19]
    ene = np.roll(ene, -4, axis = 0)
    ene = ene.reshape(9*19)
    return ene

def extract_features_Xy_cv(merged_df, allele):
    allele_data = merged_df[merged_df["allele"]==allele]
    allele_data["enefeat"] = allele_data["energies"].apply(get_energies)
    allele_data = allele_data.reset_index(drop=True)
    flag = 0
    for index, row in allele_data.iterrows():
        if flag == 0:
            X = np.array(row['enefeat'])
            flag = 1
        else: 
            X = np.vstack((X, row['enefeat']))
    #extract binding energies        
    y = np.array(list(allele_data["ba"]))
    y_l = np.array(list(allele_data["binder"]))
    
    cv_iter = []
    for split in range(5):
        test_ind = allele_data.index[(allele_data['fold_num'] == split)].tolist()
        train_ind = allele_data.index[~(allele_data['fold_num'] == split)].tolist()
        cv_iter.append((train_ind, test_ind))
        
    return (X, y, y_l, cv_iter)
    


In [22]:
def param_tune_allele(allele, train_dataset):
    
    allele_td = train_dataset[train_dataset["allele"]==allele]
    (X_train, y_train, y_l, cv) = extract_features_Xy_cv(allele_td, allele)

    grid_params = {'C': [1e-2, 1, 10], 
            'kernel': ["linear", "rbf"], 
            'degree': [3, 5, 7],
                   
            }
    
    regr_results = {}
    best_cv_mscore = 0
    best_cv_scores = None
    best_cv_params = None
    regr_svr = SVR()
    
    for i, g in enumerate(ParameterSampler(grid_params, n_iter=60)):
        print("CV")
        print(g)
        #cross-validation
        regr_svr.set_params(**g)
        regr = make_pipeline(StandardScaler(), regr_svr)
        cv_scores = cross_val_score(regr, X_train, y_train, cv=cv, n_jobs = -1)
        cv_mscore = statistics.mean(cv_scores)
        print(cv_scores)
        print(cv_mscore)
        if cv_mscore > best_cv_mscore:
            best_cv_params = g
            best_cv_scores = cv_scores
  
    print("Best CV "+str(best_cv_mscore))
    print(best_cv_params)
    return (None, None, best_cv_params, best_cv_scores)
            
    
'''
    regr_oob = RandomForestRegressor(n_jobs=-1)
    regr_cv = RandomForestRegressor(n_jobs=-1)

    best_oob_score = 0 
    best_oob_params = None
    best_cv_mscore = 0
    best_cv_scores = None
    best_cv_params = None
    for i, g in enumerate(ParameterSampler(grid_params, n_iter=100)):
        print("Parameter iteration: "+str(i))
        print("OOB")
        #out of bag
        print(g)
        regr_oob.set_params(**g)
        regr_oob.fit(X_train_s,y_train_s)
        print(regr_oob.oob_score_)
        if regr_oob.oob_score_ > best_oob_score:
            best_oob_params = g
            best_oob_score = regr_oob.oob_score_
        
        print("CV")
        #cross-validation
        regr_cv.set_params(**g)
        cv_scores = cross_val_score(regr_cv, X_train, y_train, cv=cv, n_jobs = -1)
        cv_mscore = statistics.mean(cv_scores)
        print(cv_scores)
        print(cv_mscore)
        if cv_mscore > best_cv_mscore:
            best_cv_params = g
            best_cv_scores = cv_scores
            best_cv_mscore = cv_mscore
            
    print("Best OOB "+str(best_oob_score))
    print(best_oob_params)
    print("Best CV "+str(best_cv_mscore))
    print(best_cv_params)
    return (best_oob_params, best_oob_score, best_cv_params, best_cv_scores)

'''

'\n    regr_oob = RandomForestRegressor(n_jobs=-1)\n    regr_cv = RandomForestRegressor(n_jobs=-1)\n\n    best_oob_score = 0 \n    best_oob_params = None\n    best_cv_mscore = 0\n    best_cv_scores = None\n    best_cv_params = None\n    for i, g in enumerate(ParameterSampler(grid_params, n_iter=100)):\n        print("Parameter iteration: "+str(i))\n        print("OOB")\n        #out of bag\n        print(g)\n        regr_oob.set_params(**g)\n        regr_oob.fit(X_train_s,y_train_s)\n        print(regr_oob.oob_score_)\n        if regr_oob.oob_score_ > best_oob_score:\n            best_oob_params = g\n            best_oob_score = regr_oob.oob_score_\n        \n        print("CV")\n        #cross-validation\n        regr_cv.set_params(**g)\n        cv_scores = cross_val_score(regr_cv, X_train, y_train, cv=cv, n_jobs = -1)\n        cv_mscore = statistics.mean(cv_scores)\n        print(cv_scores)\n        print(cv_mscore)\n        if cv_mscore > best_cv_mscore:\n            best_cv_params 

## Crossvalidation

In [23]:
alleles = train_dataset["allele"].unique()
results = {"allele":[], "best_oob_param":[], "best_oob_score":[], "best_cv_param":[], "best_cv_scores":[]}

for allele in alleles:
    print("------------------------------------------------------------------------")
    print("ALLELE")
    print(allele)
    res = param_tune_allele(allele, train_dataset)
    results["allele"].append(allele)
    results["best_oob_param"].append(res[0])
    results["best_oob_score"].append(res[1])
    results["best_cv_param"].append(res[2])
    results["best_cv_scores"].append(res[3])
    train_best_model(allele, res[2], "ppp")   

------------------------------------------------------------------------
ALLELE
A0101
CV
{'kernel': 'linear', 'degree': 3, 'C': 0.01}


/conda/envs/apegen/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 18 is smaller than n_iter=60. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


[0.67209626 0.75413053 0.69391555 0.69255731 0.65171135]
0.6928821997753976
CV
{'kernel': 'rbf', 'degree': 3, 'C': 0.01}
[0.55903089 0.62856057 0.57230845 0.58664457 0.59852334]
0.5890135639251914
CV
{'kernel': 'linear', 'degree': 5, 'C': 0.01}
[0.67209626 0.75413053 0.69391555 0.69255731 0.65171135]
0.6928821997753976
CV
{'kernel': 'rbf', 'degree': 5, 'C': 0.01}
[0.55903089 0.62856057 0.57230845 0.58664457 0.59852334]
0.5890135639251914
CV
{'kernel': 'linear', 'degree': 7, 'C': 0.01}
[0.67209626 0.75413053 0.69391555 0.69255731 0.65171135]
0.6928821997753976
CV
{'kernel': 'rbf', 'degree': 7, 'C': 0.01}
[0.55903089 0.62856057 0.57230845 0.58664457 0.59852334]
0.5890135639251914
CV
{'kernel': 'linear', 'degree': 3, 'C': 1}
[0.66757245 0.75311478 0.69343655 0.69298251 0.65351409]
0.6921240747899696
CV
{'kernel': 'rbf', 'degree': 3, 'C': 1}
[0.67891497 0.74333672 0.70935813 0.72377784 0.74174562]
0.7194266561878558
CV
{'kernel': 'linear', 'degree': 5, 'C': 1}
[0.66757245 0.75311478 0.6934

/conda/envs/apegen/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 18 is smaller than n_iter=60. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


CV
{'kernel': 'linear', 'degree': 3, 'C': 0.01}
[0.66504941 0.64615058 0.62535164 0.65931394 0.64448001]
0.6480691186472444
CV
{'kernel': 'rbf', 'degree': 3, 'C': 0.01}
[0.55729867 0.54429164 0.53892199 0.5540556  0.54087458]
0.5470884983750306
CV
{'kernel': 'linear', 'degree': 5, 'C': 0.01}
[0.66504941 0.64615058 0.62535164 0.65931394 0.64448001]
0.6480691186472444
CV
{'kernel': 'rbf', 'degree': 5, 'C': 0.01}
[0.55729867 0.54429164 0.53892199 0.5540556  0.54087458]
0.5470884983750306
CV
{'kernel': 'linear', 'degree': 7, 'C': 0.01}
[0.66504941 0.64615058 0.62535164 0.65931394 0.64448001]
0.6480691186472444
CV
{'kernel': 'rbf', 'degree': 7, 'C': 0.01}
[0.55729867 0.54429164 0.53892199 0.5540556  0.54087458]
0.5470884983750306
CV
{'kernel': 'linear', 'degree': 3, 'C': 1}
[0.6659006  0.64604409 0.6252211  0.65893109 0.64511754]
0.6482428805279048
CV
{'kernel': 'rbf', 'degree': 3, 'C': 1}
[0.66129375 0.64665134 0.64018664 0.65961897 0.64660568]
0.6508712745175759
CV
{'kernel': 'linear', 'd

...

CV
{'kernel': 'linear', 'degree': 3, 'C': 0.01}
[0.70054886 0.7498595  0.56458811 0.71854967 0.73167813]
0.6930448556488015
CV
{'kernel': 'rbf', 'degree': 3, 'C': 0.01}
[0.40367421 0.41407105 0.36578197 0.39904923 0.39447918]
0.3954111301422543
CV
{'kernel': 'linear', 'degree': 5, 'C': 0.01}
[0.70054886 0.7498595  0.56458811 0.71854967 0.73167813]
0.6930448556488015
CV
{'kernel': 'rbf', 'degree': 5, 'C': 0.01}
[0.40367421 0.41407105 0.36578197 0.39904923 0.39447918]
0.3954111301422543
CV
{'kernel': 'linear', 'degree': 7, 'C': 0.01}
[0.70054886 0.7498595  0.56458811 0.71854967 0.73167813]
0.6930448556488015
CV
{'kernel': 'rbf', 'degree': 7, 'C': 0.01}
[0.40367421 0.41407105 0.36578197 0.39904923 0.39447918]
0.3954111301422543
CV
{'kernel': 'linear', 'degree': 3, 'C': 1}
[0.70751346 0.73955161 0.32106012 0.7108485  0.59842128]
0.6154789950118772
CV
{'kernel': 'rbf', 'degree': 3, 'C': 1}
[0.70555935 0.73545374 0.68143261 0.695572   0.71170699]
0.7059449399772947
CV
{'kernel': 'linear', 'd

/conda/envs/apegen/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 18 is smaller than n_iter=60. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


CV
{'kernel': 'linear', 'degree': 3, 'C': 0.01}
[0.79683542 0.79358481 0.80715164 0.79055003 0.72048923]
0.7817222255595833
CV
{'kernel': 'rbf', 'degree': 3, 'C': 0.01}
[0.64700959 0.61857364 0.61778449 0.63088665 0.60736533]
0.6243239395072814
CV
{'kernel': 'linear', 'degree': 5, 'C': 0.01}
[0.79683542 0.79358481 0.80715164 0.79055003 0.72048923]
0.7817222255595833
CV
{'kernel': 'rbf', 'degree': 5, 'C': 0.01}
[0.64700959 0.61857364 0.61778449 0.63088665 0.60736533]
0.6243239395072814
CV
{'kernel': 'linear', 'degree': 7, 'C': 0.01}
[0.79683542 0.79358481 0.80715164 0.79055003 0.72048923]
0.7817222255595833
CV
{'kernel': 'rbf', 'degree': 7, 'C': 0.01}
[0.64700959 0.61857364 0.61778449 0.63088665 0.60736533]
0.6243239395072814
CV
{'kernel': 'linear', 'degree': 3, 'C': 1}
[0.7967037  0.7891629  0.80487006 0.79150752 0.62795783]
0.762040401702248
CV
{'kernel': 'rbf', 'degree': 3, 'C': 1}
[0.80344563 0.79480268 0.79312691 0.79385455 0.75870478]
0.7887869109319378
CV
{'kernel': 'linear', 'de

/conda/envs/apegen/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 18 is smaller than n_iter=60. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


CV
{'kernel': 'linear', 'degree': 3, 'C': 0.01}
[0.62777685 0.71075077 0.60825517 0.6422321  0.56745082]
0.6312931435544682
CV
{'kernel': 'rbf', 'degree': 3, 'C': 0.01}
[0.42862782 0.48896798 0.45951448 0.46520678 0.48593708]
0.4656508287503232
CV
{'kernel': 'linear', 'degree': 5, 'C': 0.01}
[0.62777685 0.71075077 0.60825517 0.6422321  0.56745082]
0.6312931435544682
CV
{'kernel': 'rbf', 'degree': 5, 'C': 0.01}
[0.42862782 0.48896798 0.45951448 0.46520678 0.48593708]
0.4656508287503232
CV
{'kernel': 'linear', 'degree': 7, 'C': 0.01}
[0.62777685 0.71075077 0.60825517 0.6422321  0.56745082]
0.6312931435544682
CV
{'kernel': 'rbf', 'degree': 7, 'C': 0.01}
[0.42862782 0.48896798 0.45951448 0.46520678 0.48593708]
0.4656508287503232
CV
{'kernel': 'linear', 'degree': 3, 'C': 1}
[0.61279487 0.71685718 0.59814709 0.62232474 0.54879717]
0.6197842089273486
CV
{'kernel': 'rbf', 'degree': 3, 'C': 1}
[0.65025628 0.71445351 0.68812096 0.67366248 0.69471245]
0.6842411370174695
CV
{'kernel': 'linear', 'd

In [14]:
results_df = pd.DataFrame(results)
results_df.to_pickle("crossval_ppp_SVM.pkl")
results_df.to_csv("crossval_ppp_SVM.csv")
results_df

,allele,best_oob_param,best_oob_score,best_cv_param,best_cv_scores
0,A0101,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6790072750194192, 0.742506610194608, 0.7089..."
1,A0201,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6595062809934203, 0.6425391447861322, 0.634..."
2,A0203,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6664562333875743, 0.7052010252373623, 0.665..."
3,A0206,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6528505766183244, 0.6726620997625883, 0.624..."
4,A0301,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6404292321364998, 0.6534728938990689, 0.633..."
5,A1101,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.7485388416399097, 0.7217285741643072, 0.716..."
6,A2301,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6239593329119871, 0.650831470784877, 0.6143..."
7,A2402,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.5495284267571928, 0.5255582522514299, 0.502..."
8,A2601,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.6148154327942268, 0.6293880423104752, 0.598..."
9,A2902,None,None,"{'kernel': 'rbf', 'degree': 7, 'C': 10}","[0.5830385522697548, 0.6444427882932677, 0.632..."


## Tune the best

In [17]:
def train_best_model(allele, params, exp_name):
    allele_train_dataset = train_dataset[train_dataset["allele"]==allele]
    (X_train, y_train, y_l, cv) = extract_features_Xy_cv(allele_train_dataset, allele)
    regr_best = SVR()
    regr_best.set_params(**params)
    regr = make_pipeline(StandardScaler(), regr_best)
    regr.fit(X_train, y_train)
    with open('./final_SVM_REGRmodels/'+allele+exp_name+'.pkl', 'wb') as fid:
        cPickle.dump(regr, fid)    

In [16]:
results_df = pd.DataFrame(results)
for allele in alleles:
    params = list(results_df[results_df["allele"]==allele]["best_cv_param"])[0]
    train_best_model(allele, params, "ppp")   

ValueError: Invalid parameter C for estimator KernelRidge(alpha=1, coef0=1, degree=7, gamma=None, kernel='rbf',
            kernel_params=None). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
results_df

## --------------------------------------------------- ##
## The middle/anchor position experiments are done for RF only 

## Middle position

In [13]:
# extracting features in training format
# and get the cross-validation iterator
def get_energies(X):
    ene = np.roll(X, -3, axis = 0)[:4,:19]
    ene = ene.reshape(4*19)
    return ene

In [ ]:
alleles = train_dataset["allele"].unique()
results = {"allele":[], "best_oob_param":[], "best_oob_score":[], "best_cv_param":[], "best_cv_scores":[]}

for allele in alleles:
    print("------------------------------------------------------------------------")
    print("ALLELE")
    print(allele)
    res = param_tune_allele(allele, train_dataset)
    results["allele"].append(allele)
    results["best_oob_param"].append(res[0])
    results["best_oob_score"].append(res[1])
    results["best_cv_param"].append(res[2])
    results["best_cv_scores"].append(res[3])

In [ ]:
def train_best_model(allele, params, exp_name):
    allele_train_dataset = train_dataset[train_dataset["allele"]==allele]
    (X_train, y_train, y_l, cv) = extract_features_Xy_cv(allele_train_dataset, allele)
    regr_best = RandomForestRegressor(n_jobs=-1)
    regr_best.set_params(**params)
    regr_best.fit(X_train, y_train)
    with open('./final_REGRmodels/'+allele+exp_name+'.pkl', 'wb') as fid:
        cPickle.dump(regr_best, fid)    

In [ ]:
for allele in alleles:
    params = list(results_df[results_df["allele"]==allele]["best_cv_param"])[0]
    train_best_model(allele, params, "ppp-middle") 

## Anchor positions

In [14]:
# extracting features in training format
# and get the cross-validation iterator
def get_energies(X):
    ene = np.roll(X, 2, axis = 0)[:5,:19]
    ene = np.roll(ene, -2, axis = 0)
    ene = ene.reshape(5*19)
    return ene

In [ ]:
alleles = train_dataset["allele"].unique()
results = {"allele":[], "best_oob_param":[], "best_oob_score":[], "best_cv_param":[], "best_cv_scores":[]}

for allele in alleles:
    print("------------------------------------------------------------------------")
    print("ALLELE")
    print(allele)
    res = param_tune_allele(allele, train_dataset)
    results["allele"].append(allele)
    results["best_oob_param"].append(res[0])
    results["best_oob_score"].append(res[1])
    results["best_cv_param"].append(res[2])
    results["best_cv_scores"].append(res[3])

In [ ]:
def train_best_model(allele, params, exp_name):
    allele_train_dataset = train_dataset[train_dataset["allele"]==allele]
    (X_train, y_train, y_l, cv) = extract_features_Xy_cv(allele_train_dataset, allele)
    regr_best = RandomForestRegressor(n_jobs=-1)
    regr_best.set_params(**params)
    regr_best.fit(X_train, y_train)
    with open('./final_REGRmodels/'+allele+exp_name+'.pkl', 'wb') as fid:
        cPickle.dump(regr_best, fid)

In [ ]:
for allele in alleles:
    params = list(results_df[results_df["allele"]==allele]["best_cv_param"])[0]
    train_best_model(allele, params, "ppp-anchor") 